# <center> 2차 크롤링 - <점핏> </center>

##### 링크들 들어가서 각각 정보 수집, 자율주행

* 기업이름
* 고용형태 : 신입, 경력
* 연봉 (대체로 면접후 결정 일겁니다, 크롤링 사이트 내용 그대로 검출하면 될듯!)
* 기업주소 : 기업 메인 위치
* 키워드 : 자율주행 / 드론 / 딥러닝 / 로봇 제어</br></br>
 -----뭉쳐서 '주요사항' column-----
* 자격 요건 : ROS, Python, C++, 학력
* 담당업무
* 우대사항

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service("../drive/chromedriver-linux64/chromedriver"), options=options)

driver.get("https://www.jumpit.co.kr/search?sort=relation&keyword=%EC%9E%90%EC%9C%A8%EC%A3%BC%ED%96%89")
time.sleep(2)

# 페이지 끝까지 스크롤
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

html_source = driver.page_source
soup = BeautifulSoup(html_source, "html.parser")
time.sleep(3)

job_box_1 = soup.find('section', class_='sc-c12e57e5-3 gjgpzi')
job_box_2 = job_box_1.find('section', class_='sc-80cff0b9-0 cOzTle')
job_box_3 = job_box_2.find_all('div', class_='sc-c8169e0e-0 gOSXGP')

image_link_list = []
category_list = []
major_work_list =[]

for info in job_box_3:

    job_link = info.find('a')['href']
    image_link_list.append(job_link)

    major_work = info.find('h2', class_='position_card_info_title').text
    major_work_list.append(major_work)
    category_list.append('자율주행')

print(major_work_list, image_link_list, category_list)


In [19]:
print(len(image_link_list))
print(len(category_list))

139
139


In [20]:
import pandas as pd

df = pd.DataFrame({
    '키워드': category_list,
    'url': image_link_list,
    '담당업무' : major_work_list
})

In [21]:
df.tail()

,키워드,url,담당업무
134,자율주행,/position/16281,디지털 H/W 회로설계 개발 경력직 채용(RADAR)
135,자율주행,/position/22356,무인운반차(AGV) 주행 분야 SW개발자 모집
136,자율주행,/position/11779,백엔드 엔지니어 (AI 로봇 관제 솔루션)
137,자율주행,/position/21509,디지털 트윈 제작을 위한 Unity 3D 개발자
138,자율주행,/position/21510,디지털 트윈 제작을 위한 Unity 3D 모델러


In [22]:
df.to_csv("../data/jumpit_base.csv", sep =",", encoding = 'utf-8')

In [23]:
import pandas as pd

df = pd.read_csv("../data/jumpit_base.csv", encoding='utf-8', index_col=0)

df['url'] = df['url'].apply(lambda x: f"https://www.jumpit.co.kr{x}")


In [1]:
import pandas as pd
df= pd.read_csv("../data/jumpit_base.csv", encoding='utf-8', index_col=0)

In [24]:
df.tail()

,키워드,url,담당업무
134,자율주행,https://www.jumpit.co.kr/position/16281,디지털 H/W 회로설계 개발 경력직 채용(RADAR)
135,자율주행,https://www.jumpit.co.kr/position/22356,무인운반차(AGV) 주행 분야 SW개발자 모집
136,자율주행,https://www.jumpit.co.kr/position/11779,백엔드 엔지니어 (AI 로봇 관제 솔루션)
137,자율주행,https://www.jumpit.co.kr/position/21509,디지털 트윈 제작을 위한 Unity 3D 개발자
138,자율주행,https://www.jumpit.co.kr/position/21510,디지털 트윈 제작을 위한 Unity 3D 모델러


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import requests
import os
from tqdm.notebook import tqdm


options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={generate_user_agent()}") 

driver = webdriver.Chrome(service=Service("../drive/chromedriver-linux64/chromedriver"),options=options)


company_address_list = []
company_name_list=[]
hire_list = []
major_work_list =[]
tech_list=[]
salary_list =[]


for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    driver.get(row['url'] )
    major_work_list_tmp = []

    time.sleep(1)  

    for _ in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, "html.parser")

    name_box = soup.find('section', class_='sc-3a5c981a-0 iORPxB')
    company_name = name_box.find('a').text
    company_name_list.append(company_name)

    company_address_box = soup.find('section', class_='sc-2da322c6-0 hWFbtF')
    company_address_box2 = company_address_box.find_all('dl', class_='sc-2da322c6-1 ksYhFT')

    company_address_list.append(company_address_box2[3].text)
    hire_list.append(company_address_box2[0].text)

    essential_tech_box = soup.find_all('dl', class_='sc-ae955bd-0 MHqtB')

    salary_list.append('회사내규에 따름')

    tech_stack = essential_tech_box[0].find('pre').text
    major_work_list_tmp.append(tech_stack)

    essential_tech1 = essential_tech_box[1].find('pre').text
    major_work_list_tmp.append(essential_tech1)


    essential_tech2 = essential_tech_box[3].find('pre').text
    major_work_list_tmp.append(essential_tech2)
    
    
    tech_list.append(major_work_list_tmp)
    
    

In [ ]:
print(len(company_name_list))
print(len(hire_list))
print(len(tech_list))
print(len(company_address_list))

In [10]:
import pandas as pd

# 새로운 DataFrame 생성
new_data = pd.DataFrame({
    
    '기업이름' : company_name_list,
    '고용형태': hire_list,
    '연봉' : salary_list,
    '기업주소': company_address_list,
    
    '필요역량':tech_list
})

# 기존의 CSV 파일 불러오기
job_data = pd.read_csv('../data/jumpit_base.csv')

# 두 DataFrame 병합
merged_data = pd.concat([job_data, new_data], axis=1)


# 결과 저장
merged_data.to_csv('../data/jumpit_basic2.csv', index=False)


In [11]:
merged_data.head()


,Unnamed: 0,키워드,url,기업이름,고용형태,연봉,기업주소,담당업무,필요역량
0,0,자율주행,/position/22380,베스텔라랩,경력경력 2~15년,회사내규에 따름,근무지역대구 동구 동대구로 465 대구스케일업허브지도보기·주소복사경기 안양시 동안구...,• 자율주행차량 제어 프로그램 개발\n• 자사 측위 알고리즘 개선 및 엔진 고도화\...,"[Spring FrameworkGitRiOSROSC++AWSPythonSQL, • ..."
1,1,자율주행,/position/13806,긴트,경력경력 2~9년,회사내규에 따름,근무지역경기 성남시 분당구 판교로228번길 17 4층 긴트지도보기·주소복사,• 농기계 자율주행 및 AI 관련 해외 논문 조사 및 요약\n• 농경지 자율주행을 ...,"[C++SWMATLABPythonC, • 기계, 컴퓨터, 자동차, 제어, 로봇공학 ..."
2,2,자율주행,/position/8818,에이알247,경력경력 1~10년,회사내규에 따름,근무지역경기도 성남시 분당구 구미동 로드랜드이지타워 108호지도보기·주소복사,• 자율주행 무인 배달 로봇 위치추정 및 지도제작\n• Lidar SLAM / Vi...,"[C++ROSPythonC, • SLAM 을 활용한 위치추정 개발 경력\n• 자율주..."
3,3,자율주행,/position/10634,트리즈엔지니어링,경력경력 2~10년,회사내규에 따름,근무지역경기도 화성시 동탄산단5길 10-12지도보기·주소복사,"• 실내·외 로봇 인지, 판단, 제어 시스템 SW 개발\n• ROS, Linux, ...","[ROSLinuxC++SWPythonCC#Windows, • Python, C, C..."
4,4,자율주행,/position/8690,알지티,경력경력 3~7년,회사내규에 따름,근무지역서울 금천구 디지털로10길 78 10층 1009호지도보기·주소복사,• 당사제품에 대한 펌웨어 개발\n• 당사제품에 대한 하드웨어 설계 및 검증\n• ...,"[OrcadC++FWPythonC, • 자율주행로봇 관련분야에서의 프로젝트 경험을 ..."


In [12]:
# 컬럼 재정렬 및 불필요한 컬럼 제거
merged_data = merged_data[['기업이름', '고용형태', '연봉', '기업주소', '키워드', '담당업무','필요역량']]

merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,베스텔라랩,경력경력 2~15년,회사내규에 따름,근무지역대구 동구 동대구로 465 대구스케일업허브지도보기·주소복사경기 안양시 동안구...,자율주행,• 자율주행차량 제어 프로그램 개발\n• 자사 측위 알고리즘 개선 및 엔진 고도화\...,"[Spring FrameworkGitRiOSROSC++AWSPythonSQL, • ..."
1,긴트,경력경력 2~9년,회사내규에 따름,근무지역경기 성남시 분당구 판교로228번길 17 4층 긴트지도보기·주소복사,자율주행,• 농기계 자율주행 및 AI 관련 해외 논문 조사 및 요약\n• 농경지 자율주행을 ...,"[C++SWMATLABPythonC, • 기계, 컴퓨터, 자동차, 제어, 로봇공학 ..."
2,에이알247,경력경력 1~10년,회사내규에 따름,근무지역경기도 성남시 분당구 구미동 로드랜드이지타워 108호지도보기·주소복사,자율주행,• 자율주행 무인 배달 로봇 위치추정 및 지도제작\n• Lidar SLAM / Vi...,"[C++ROSPythonC, • SLAM 을 활용한 위치추정 개발 경력\n• 자율주..."
3,트리즈엔지니어링,경력경력 2~10년,회사내규에 따름,근무지역경기도 화성시 동탄산단5길 10-12지도보기·주소복사,자율주행,"• 실내·외 로봇 인지, 판단, 제어 시스템 SW 개발\n• ROS, Linux, ...","[ROSLinuxC++SWPythonCC#Windows, • Python, C, C..."
4,알지티,경력경력 3~7년,회사내규에 따름,근무지역서울 금천구 디지털로10길 78 10층 1009호지도보기·주소복사,자율주행,• 당사제품에 대한 펌웨어 개발\n• 당사제품에 대한 하드웨어 설계 및 검증\n• ...,"[OrcadC++FWPythonC, • 자율주행로봇 관련분야에서의 프로젝트 경험을 ..."


In [13]:
# 결과 저장
merged_data.to_csv('../data/project_eda_dataset/jumpit_autonomous.csv', index=False)

In [14]:
merged_data['기업주소'] = merged_data['기업주소'].apply(lambda x: x.split('지도보기·주소복사')[0].replace('근무지역', '').strip())

# 결과 확인 및 저장
merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,베스텔라랩,경력경력 2~15년,회사내규에 따름,대구 동구 동대구로 465 대구스케일업허브,자율주행,• 자율주행차량 제어 프로그램 개발\n• 자사 측위 알고리즘 개선 및 엔진 고도화\...,"[Spring FrameworkGitRiOSROSC++AWSPythonSQL, • ..."
1,긴트,경력경력 2~9년,회사내규에 따름,경기 성남시 분당구 판교로228번길 17 4층 긴트,자율주행,• 농기계 자율주행 및 AI 관련 해외 논문 조사 및 요약\n• 농경지 자율주행을 ...,"[C++SWMATLABPythonC, • 기계, 컴퓨터, 자동차, 제어, 로봇공학 ..."
2,에이알247,경력경력 1~10년,회사내규에 따름,경기도 성남시 분당구 구미동 로드랜드이지타워 108호,자율주행,• 자율주행 무인 배달 로봇 위치추정 및 지도제작\n• Lidar SLAM / Vi...,"[C++ROSPythonC, • SLAM 을 활용한 위치추정 개발 경력\n• 자율주..."
3,트리즈엔지니어링,경력경력 2~10년,회사내규에 따름,경기도 화성시 동탄산단5길 10-12,자율주행,"• 실내·외 로봇 인지, 판단, 제어 시스템 SW 개발\n• ROS, Linux, ...","[ROSLinuxC++SWPythonCC#Windows, • Python, C, C..."
4,알지티,경력경력 3~7년,회사내규에 따름,서울 금천구 디지털로10길 78 10층 1009호,자율주행,• 당사제품에 대한 펌웨어 개발\n• 당사제품에 대한 하드웨어 설계 및 검증\n• ...,"[OrcadC++FWPythonC, • 자율주행로봇 관련분야에서의 프로젝트 경험을 ..."


In [15]:
merged_data['고용형태'] = merged_data['고용형태'].str.replace('경력경력', '경력')

In [16]:
merged_data.head()

,기업이름,고용형태,연봉,기업주소,키워드,담당업무,필요역량
0,베스텔라랩,경력 2~15년,회사내규에 따름,대구 동구 동대구로 465 대구스케일업허브,자율주행,• 자율주행차량 제어 프로그램 개발\n• 자사 측위 알고리즘 개선 및 엔진 고도화\...,"[Spring FrameworkGitRiOSROSC++AWSPythonSQL, • ..."
1,긴트,경력 2~9년,회사내규에 따름,경기 성남시 분당구 판교로228번길 17 4층 긴트,자율주행,• 농기계 자율주행 및 AI 관련 해외 논문 조사 및 요약\n• 농경지 자율주행을 ...,"[C++SWMATLABPythonC, • 기계, 컴퓨터, 자동차, 제어, 로봇공학 ..."
2,에이알247,경력 1~10년,회사내규에 따름,경기도 성남시 분당구 구미동 로드랜드이지타워 108호,자율주행,• 자율주행 무인 배달 로봇 위치추정 및 지도제작\n• Lidar SLAM / Vi...,"[C++ROSPythonC, • SLAM 을 활용한 위치추정 개발 경력\n• 자율주..."
3,트리즈엔지니어링,경력 2~10년,회사내규에 따름,경기도 화성시 동탄산단5길 10-12,자율주행,"• 실내·외 로봇 인지, 판단, 제어 시스템 SW 개발\n• ROS, Linux, ...","[ROSLinuxC++SWPythonCC#Windows, • Python, C, C..."
4,알지티,경력 3~7년,회사내규에 따름,서울 금천구 디지털로10길 78 10층 1009호,자율주행,• 당사제품에 대한 펌웨어 개발\n• 당사제품에 대한 하드웨어 설계 및 검증\n• ...,"[OrcadC++FWPythonC, • 자율주행로봇 관련분야에서의 프로젝트 경험을 ..."


In [17]:
merged_data.to_csv('../data/project_eda_dataset/jumpit_autonomous.csv', index=False)